# 07-08: Google Gemini API使用 Google Gemini API 进行文本生成和对话。

In [ ]:
// 安装: npm install @google/generative-ai
import { GoogleGenerativeAI } from '@google/generative-ai';

const genAI = new GoogleGenerativeAI(process.env.GEMINI_API_KEY || '');

// ========== 1. 基础文本生成 ==========
async function basicGeneration() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro' });

  const prompt = 'Write a story about a magic backpack.';
  const result = await model.generateContent(prompt);
  const response = await result.response;
  const text = response.text();

  console.log(text);
  return text;
}

In [ ]:
// ========== 2. 多轮对话 ==========
async function chatConversation() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro' });
  const chat = model.startChat({
    history: [
      {
        role: 'user',
        parts: 'Hello, I have 2 dogs in my house.'
      },
      {
        role: 'model',
        parts: 'Great to meet you. What would you like to know?'
      }
    ],
    generationConfig: {
      maxOutputTokens: 100
    }
  });

  const msg = 'How many paws are in my house?';
  const result = await chat.sendMessage(msg);
  const response = await result.response;

  console.log(response.text());
  // Output: If you have 2 dogs, and each dog has 4 paws, then there are 8 paws in your house.
}

// 继续对话
async function continueChat(chat: any) {
  const result = await chat.sendMessage('Tell me more about dogs.');
  const response = await result.response;
  console.log(response.text());
}

In [ ]:
// ========== 3. 流式响应 ==========
async function streamingGeneration() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro' });

  const prompt = 'Write a long story about space exploration.';
  const result = await model.generateContentStream(prompt);

  for await (const chunk of result.stream) {
    const chunkText = chunk.text();
    process.stdout.write(chunkText);
  }
}

// 流式对话
async function streamingChat() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro' });
  const chat = model.startChat();

  const result = await chat.sendMessageStream(
    'Tell me a joke about programming'
  );

  for await (const chunk of result.stream) {
    process.stdout.write(chunk.text());
  }
}

In [ ]:
// ========== 4. 图像理解 (Gemini Pro Vision) ==========
import { readFileSync } from 'fs';

async function imageUnderstanding() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro-vision' });

  const imagePath = './image.png';
  const imageData = readFileSync(imagePath);
  const base64Image = imageData.toString('base64');

  const result = await model.generateContent([
    'What is in this image?',
    {
      inlineData: {
        data: base64Image,
        mimeType: 'image/png'
      }
    }
  ]);

  const response = await result.response;
  console.log(response.text());
}

// 多图像输入
async function multiImageInput() {
  const model = genAI.getGenerativeModel({ model: 'gemini-pro-vision' });

  const image1 = readFileSync('./image1.png').toString('base64');
  const image2 = readFileSync('./image2.png').toString('base64');

  const result = await model.generateContent([
    'Compare these two images:',
    {
      inlineData: {
        data: image1,
        mimeType: 'image/png'
      }
    },
    {
      inlineData: {
        data: image2,
        mimeType: 'image/png'
      }
    }
  ]);

  const response = await result.response;
  console.log(response.text());
}

In [ ]:
// ========== 5. 配置参数 ==========
async function withConfig() {
  const model = genAI.getGenerativeModel({
    model: 'gemini-pro',
    generationConfig: {
      temperature: 0.7,      // 创造性: 0-1
      topK: 40,             // 候选 token 数量
      topP: 0.95,           // 概率质量阈值
      maxOutputTokens: 1024 // 最大输出长度
    },
    safetySettings: [
      {
        category: 'HARM_CATEGORY_HARASSMENT',
        threshold: 'BLOCK_MEDIUM_AND_ABOVE'
      },
      {
        category: 'HARM_CATEGORY_HATE_SPEECH',
        threshold: 'BLOCK_MEDIUM_AND_ABOVE'
      }
    ]
  });

  const result = await model.generateContent(
    'Write a creative story about friendship'
  );

  return result.response.text();
}

In [ ]:
// ========== 6. 嵌入向量 ==========
async function getEmbeddings() {
  const model = genAI.getGenerativeModel({ model: 'embedding-001' });

  const result = await model.embedContent('The quick brown fox jumps over the lazy dog.');
  const embedding = result.embedding;

  console.log('Embedding values:', embedding.values.length);
  console.log('First 5 values:', embedding.values.slice(0, 5));

  return embedding;
}

// 批量嵌入
async function batchEmbeddings() {
  const model = genAI.getGenerativeModel({ model: 'embedding-001' });

  const texts = [
    'How to make a sandwich',
    'The history of bread',
    'Healthy eating habits'
  ];

  const batchResult = await model.batchEmbedContents({
    requests: texts.map(text => ({ content: { role: 'user', parts: [text] } }))
  });

  console.log('Number of embeddings:', batchResult.embeddings.length);
  return batchResult.embeddings;
}

In [ ]:
// ========== 7. 函数调用 ==========
async function functionCalling() {
  const model = genAI.getGenerativeModel({
    model: 'gemini-pro',
    tools: [
      {
        functionDeclarations: [
          {
            name: 'getWeather',
            description: 'Get the current weather in a given location',
            parameters: {
              type: 'object',
              properties: {
                location: {
                  type: 'string',
                  description: 'The city name, e.g. San Francisco'
                },
                unit: {
                  type: 'string',
                  enum: ['celsius', 'fahrenheit'],
                  description: 'Temperature unit'
                }
              },
              required: ['location']
            }
          },
          {
            name: 'getExchangeRate',
            description: 'Get the exchange rate between currencies',
            parameters: {
              type: 'object',
              properties: {
                from: {
                  type: 'string',
                  description: 'Source currency code'
                },
                to: {
                  type: 'string',
                  description: 'Target currency code'
                }
              },
              required: ['from', 'to']
            }
          }
        ]
      }
    ]
  });

  const chat = model.startChat();

  // 发送消息
  const result = await chat.sendMessage(
    'What is the weather in Tokyo and the exchange rate from USD to JPY?'
  );

  const response = result.response;

  // 检查是否有函数调用
  const functionCalls = response.functionCalls();

  if (functionCalls && functionCalls.length > 0) {
    for (const call of functionCalls) {
      console.log('Function call:', call.name, call.args);

      // 执行函数并返回结果
      const functionResult = await executeFunction(call.name, call.args);

      // 发送函数结果回模型
      const followUp = await chat.sendMessage([
        {
          functionResponse: {
            name: call.name,
            response: functionResult
          }
        }
      ]);

      console.log('Final response:', followUp.response.text());
    }
  }
}

async function executeFunction(name: string, args: any): Promise<any> {
  if (name === 'getWeather') {
    return { temperature: 25, condition: 'sunny', location: args.location };
  }
  if (name === 'getExchangeRate') {
    return { rate: 150.5, from: args.from, to: args.to };
  }
  return {};
}